In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Xsum train data 1.csv')

In [ ]:
data.head()

,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984


In [ ]:
pip install transformers[torch] accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
pip install datasets

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric

# Load the CSV file
file_path = '/mnt/data/Xsum train data 1.csv'
df = pd.read_csv('/content/drive/MyDrive/Xsum train data 1.csv')

# Display the first few rows of the DataFrame
print(df.head())

# Fill NaN values with empty strings
df.fillna("", inplace=True)


                                            document  \
0  The full cost of damage in Newton Stewart, one...   
1  A fire alarm went off at the Holiday Inn in Ho...   
2  Ferrari appeared in a position to challenge un...   
3  John Edward Bates, formerly of Spalding, Linco...   
4  Patients and staff were evacuated from Cerahpa...   

                                             summary        id  
0  Clean-up operations are continuing across the ...  35232142  
1  Two tourist buses have been destroyed by fire ...  40143035  
2  Lewis Hamilton stormed to pole position at the...  35951548  
3  A former Lincolnshire Police officer carried o...  36266422  
4  An armed man who locked himself into a room at...  38826984  


In [ ]:
# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Define the preprocess function
def preprocess_data(dataframe, source_text_column, target_text_column):
    inputs = ["summarize: " + str(doc) for doc in dataframe[source_text_column]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(dataframe[target_text_column].tolist(), max_length=128, truncation=True, padding="max_length", return_tensors="pt")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing to the DataFrame
processed_data = preprocess_data(df, 'document', 'summary')  # Adjust column names as needed

# Convert the processed data to dictionary
data_dict = {key: value.numpy() for key, value in processed_data.items()}

# Split the data
train_size = 0.9
train_dataset, test_dataset = train_test_split(df, train_size=train_size, random_state=42)

# Convert to datasets
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

# Tokenize datasets
def tokenize_function(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
! pip install datasets transformers accelerate 'transformers[torch]' -U huggingface_hub torch

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric

# Load the CSV file
file_path = '/mnt/data/Xsum train data 1.csv'
df = pd.read_csv('/content/drive/MyDrive/Xsum train data 1.csv')

# Display the first few rows of the DataFrame
print(df.head())

# Fill NaN values with empty strings
df.fillna("", inplace=True)


                                            document  \
0  The full cost of damage in Newton Stewart, one...   
1  A fire alarm went off at the Holiday Inn in Ho...   
2  Ferrari appeared in a position to challenge un...   
3  John Edward Bates, formerly of Spalding, Linco...   
4  Patients and staff were evacuated from Cerahpa...   

                                             summary        id  
0  Clean-up operations are continuing across the ...  35232142  
1  Two tourist buses have been destroyed by fire ...  40143035  
2  Lewis Hamilton stormed to pole position at the...  35951548  
3  A former Lincolnshire Police officer carried o...  36266422  
4  An armed man who locked himself into a room at...  38826984  


In [ ]:
# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Define the preprocess function
def preprocess_data(dataframe, source_text_column, target_text_column):
    inputs = ["summarize: " + str(doc) for doc in dataframe[source_text_column]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors="pt")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(dataframe[target_text_column].tolist(), max_length=128, truncation=True, padding="max_length", return_tensors="pt")

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing to the DataFrame
processed_data = preprocess_data(df, 'document', 'summary')  # Adjust column names as needed

# Convert the processed data to dictionary
data_dict = {key: value.numpy() for key, value in processed_data.items()}

# Split the data
train_size = 0.9
train_dataset, test_dataset = train_test_split(df, train_size=train_size, random_state=42)

# Convert to datasets
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

# Tokenize datasets
def tokenize_function(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Initialize the model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./abstractive_model")
tokenizer.save_pretrained("./abstractive_model")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.696500,0.620079
2,0.673400,0.611563
3,0.659400,0.609784


('./abstractive_model/tokenizer_config.json',
 './abstractive_model/special_tokens_map.json',
 './abstractive_model/spiece.model',
 './abstractive_model/added_tokens.json')

In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=6245be7d66aca20e4df5cd87e7ee7209e467989414f0256b4875d9f514a8609a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Function to generate summaries and evaluate them
def evaluate(model, tokenizer, dataset, max_input_length=512, max_target_length=128):
    model.eval()
    metric = load_metric("rouge")
    summaries = []
    references = []

    for batch in dataset:
        input_ids = batch["input_ids"].unsqueeze(0).to(model.device)
        attention_mask = batch["attention_mask"].unsqueeze(0).to(model.device)

        with torch.no_grad():
            summary_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=max_target_length)

        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        reference = tokenizer.decode(batch["labels"], skip_special_tokens=True)

        summaries.append(summary)
        references.append(reference)

    results = metric.compute(predictions=summaries, references=references, use_stemmer=True)
    return results

# Evaluate the model on the test data
results = evaluate(model, tokenizer, test_dataset)
print("ROUGE-1:", results["rouge1"].high.fmeasure)
print("ROUGE-2:", results["rouge2"].high.fmeasure)
print("ROUGE-L:", results["rougeL"].high.fmeasure)


ROUGE-1: 0.2897436474211089
ROUGE-2: 0.07612913769479013
ROUGE-L: 0.22019318663391835
